# Modelos LLMs pré-treinados com Hugging Face

Este projeto tem como objetivo apresentar o processo de pré-treinamento de modelos de linguagem grandes (LLMs) usando a popular biblioteca Hugging Face. Hugging Face é uma plataforma de código aberto líder para processamento de linguagem natural que fornece uma ampla gama de modelos pré-treinados e ferramentas para ajuste fino e implantação desses modelos.

Vamos carregar modelos pré-treinados do Hugging Face e fazer inferências usando o módulo Pipeline. Além disso, vamos treinar LLMs pré-treinados em seus próprios dados (ajuste fino autossupervisionado). 

### Objetivos

- Carregar LLMs pré-treinados do Hugging Face e fazer inferências usando o módulo pipeline
- Treinar LLMs pré-treinados em seus dados
- Armazenar LLMs para ajustá-los para casos de uso específicos

### Preparar setup

In [2]:
def warn(*args, **kwargs):
    pass
    
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import importlib.util
import subprocess
import sys

def check_and_install(package, pip_name=None):
    if pip_name is None:
        pip_name = package
    spec = importlib.util.find_spec(package)
    if spec is None:
        print(f"{package} não está instalado. Instalando...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])
    else:
        print(f"{package} já está instalado.")

In [3]:
check_and_install('torch')
check_and_install('transformers')
check_and_install('datasets')
check_and_install('portalocker', 'portalocker>=2/0.0')
check_and_install('accelerate')
check_and_install('torchvision')
check_and_install('protobuf', 'protobuf==3.20.*')

torch já está instalado.
transformers já está instalado.
datasets já está instalado.
portalocker já está instalado.
accelerate já está instalado.
torchvision já está instalado.
protobuf não está instalado. Instalando...


### Carregar bibliotecas

In [4]:
import torch
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoConfig,AutoModelForCausalLM,AutoModelForSequenceClassification,BertConfig,BertForMaskedLM,TrainingArguments, Trainer, TrainingArguments
from transformers import AutoTokenizer,BertTokenizerFast,TextDataset,DataCollatorForLanguageModeling
from transformers import pipeline
from datasets import load_dataset

from tqdm.auto import tqdm
import math
import time
import os

In [5]:
# Disable tokenizer parallelism to avoid deadlocks.
# Set the environment variable TOKENIZERS_PARALLELISM to 'false'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

### Pré-treinamento e ajuste fino auto-supervisionado

O pré-treinamento é uma técnica usada no processamento de linguagem natural (NLP) para treinar grandes modelos de linguagem (LLMs) em um vasto corpus de dados de texto não rotulados. O objetivo é capturar os padrões gerais e relacionamentos semânticos presentes na linguagem natural, permitindo que o modelo desenvolva uma compreensão profunda da estrutura e do significado da linguagem.

A motivação por trás dos transformadores de pré-treinamento é abordar as limitações das abordagens tradicionais de NLP que geralmente exigem quantidades significativas de dados rotulados para cada tarefa específica. Ao alavancar a abundância de dados de texto não rotulados, o pré-treinamento permite que o modelo aprenda habilidades fundamentais de linguagem por meio de objetivos autossupervisionados, facilitando a aprendizagem por transferência.

Os objetivos de pré-treinamento, como modelagem de linguagem mascarada (MLM) e predição da próxima frase (NSP), desempenham um papel crucial no sucesso dos modelos transformadores. Os modelos pré-treinados podem ser ajustados ainda mais treinando-os em dados não rotulados específicos do domínio, o que é conhecido como ajuste fino autossupervisionado.

Além disso, o modelo pode ser ajustado em tarefas downstream específicas usando dados rotulados, um processo conhecido como ajuste fino supervisionado, melhorando ainda mais seu desempenho.

Nas seções a seguir deste laboratório, você explorará objetivos de pré-treinamento, carregamento de modelos pré-treinados, preparação de dados e o processo de ajuste fino. Ao final, você terá uma compreensão sólida do pré-treinamento e do ajuste fino autossupervisionado, capacitando-o a aplicar essas técnicas para resolver problemas de PNL do mundo real.

In [6]:
# Loading a pretrained model from Hugging Face and making an inference
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [7]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [9]:
print(pipe("This movie was really")[0]["generated_text"])

This movie was really good. I was really surprised by how good it was.
I was


### Objetivos de pré-treinamento

Objetivos de pré-treinamento são componentes cruciais do processo de pré-treinamento para transformadores. Esses objetivos definem as tarefas nas quais o modelo é treinado durante a fase de pré-treinamento, permitindo que ele aprenda representações contextuais significativas da linguagem. Três objetivos de pré-treinamento comumente usados ​​são modelagem de linguagem mascarada (MLM), previsão da próxima frase (NSP) e previsão do próximo Ttoken.